# Supermarkets modeling

The goal of this document....

In [14]:
import pandas as pd
import numpy as np

In [15]:
# import data
supermarkets = {
    'aldinorth': pd.read_pickle('data/products-clean/aldinorth-products-clean.pkl'),
    'aldisouth': pd.read_pickle('data/products-clean/aldisouth-products-clean.pkl'),
    'edeka': pd.read_pickle('data/products-clean/edeka-products-clean.pkl')
}

In [16]:
supermarkets['aldinorth']

,Name,Price
0,Tamara extra strawberry jam,0.99
1,Nutella,3.79
2,Coke,2.89
3,Trader Joe's / Aldi Nord Lemon Iced Tea,0.49
4,Iglo fish fingers,3.49
...,...,...
2984,Moser Roth Fine Easter Eggs Fine Bitter 70%,3.65
2985,Moser Roth Fine Easter Eggs Noble Nougat,1.49
2986,Moser Roth Fine Easter Eggs Chocolate Candy,0.99
2987,Moser Roth Fine Easter Eggs Noble Collection w...,1.15


## 1. To do's

- [v0] - Build an algorithm able to (input: ingredient, output: % coverage + total price per each supermarket)
- [v1] - Build an algorithm able to (input: list of ingredients, output: % coverage + total price per each supermarket)
- [v2] - Use fuzzy buzzy to match the input (potato) with the ingredients by similarity.

### 1.1 [v0] algorithm
- [v0] - Build an algorithm able to (input: ingredient, output: % coverage + total price per each supermarket)

In [4]:
# We are looking for the price of the ingredient that we are searching per each supermarket
supermarkets['rewe'][supermarkets['rewe']['Name'] == 'Pfanner Mango Fair 100%  Fair']['Price'].values[0]

NameError: name 'supermarkets' is not defined

In [ ]:
def search(ingredient:str):
    # New empty list
    result = []
    for market in supermarkets:
        a = supermarkets[market]
        try:
            price = a[a['Name'] == ingredient]['Price'].values[0]
            result.append(f'{market}: {price}')
        except:
            result.append(f'{market}: Not found')
    return result

# Try it with 'Coca Cola' because it is available in every supermarket
# Try with 'Pfanner Mango Fair 100%  Fair'

In [ ]:
# Testing the function
search('Pfanner Mango Fair 100%  Fair')

In [ ]:
# Testing the function
search('Coca Cola')

We can se that there are some inconsistencies with the 'Coca Cola' prices. Let's find out which is the problem

In [ ]:
supermarkets['kaufland'][supermarkets['kaufland']['Name'] == 'Coca Cola']

### 1.2 [v1] algorithm
- [v1] - Build an algorithm able to (input: list of ingredients, output: % coverage + total price per each supermarket)

In [ ]:
def search_list(ingredients:list):
    # New empty list
    result = []
    
    for market in supermarkets:
        
        a = supermarkets[market]
        
        for ing in ingredients:    
            try:
                price = a[a['Name'] == ing]['Price'].values[0]
                result.append(f'{market}: {price} for the {ing}')
            except:
                result.append(f'{market}: Not found')
            
    return result

In [ ]:
search_list(['Coca Cola', 'Pfanner Mango Fair 100%  Fair'])